In [1]:
import json
from pprint import pprint
import pandas as pd
import json
import os
import numpy as np
results_dir = './bench_results/'


In [2]:
reading_files = os.listdir(os.path.join(results_dir, 'reading'))
writing_files = os.listdir(os.path.join(results_dir, 'writing'))

print(reading_files)
print(writing_files)

['bench_read_i16_as_f32_tracemalloc.json', 'bench_read_i16_timings.json', 'bench_read_i16_tracemalloc.json', 'bench_read_i16_as_f32_track_memory.json', 'bench_read_i16_as_f32_timings.json', 'bench_read_f32_as_i16_timings.json', 'bench_read_f32_as_i16_tracemalloc.json', 'bench_read_f32_as_i16_track_memory.json', 'bench_read_i16_track_memory.json']
['bench_write_i16_as_f32_timings.json', 'bench_write_f32_timings.json', 'bench_write_i16_tracemalloc.json', 'bench_write_i16_as_f32_track_memory.json', 'bench_write_i16_as_f32_tracemalloc.json', 'bench_write_i16_track_memory.json', 'bench_write_f32_tracemalloc.json', 'bench_write_f32_track_memory.json', 'bench_write_i16_timings.json']


In [3]:
# strip the bench part from the file names
# reading_files = [f.replace('bench_read_', '') for f in reading_files]
# writing_files = [f.replace('bench_write_', '') for f in writing_files]
print(reading_files)
print(writing_files)

['bench_read_i16_as_f32_tracemalloc.json', 'bench_read_i16_timings.json', 'bench_read_i16_tracemalloc.json', 'bench_read_i16_as_f32_track_memory.json', 'bench_read_i16_as_f32_timings.json', 'bench_read_f32_as_i16_timings.json', 'bench_read_f32_as_i16_tracemalloc.json', 'bench_read_f32_as_i16_track_memory.json', 'bench_read_i16_track_memory.json']
['bench_write_i16_as_f32_timings.json', 'bench_write_f32_timings.json', 'bench_write_i16_tracemalloc.json', 'bench_write_i16_as_f32_track_memory.json', 'bench_write_i16_as_f32_tracemalloc.json', 'bench_write_i16_track_memory.json', 'bench_write_f32_tracemalloc.json', 'bench_write_f32_track_memory.json', 'bench_write_i16_timings.json']


In [4]:
def split_int(files):
    trace_mallocs = []
    track_memory = []
    timings = []
    for f in files:
        if 'tracemalloc' in f:
            trace_mallocs.append(f)
        elif 'track_memory' in f:
            track_memory.append(f)
        else:
            timings.append(f)
    return timings, trace_mallocs, track_memory

In [5]:
r_timings, r_mallocs, r_memory = split_int(reading_files)
w_timings, w_mallocs, w_memory = split_int(writing_files)
print(r_timings)

['bench_read_i16_timings.json', 'bench_read_i16_as_f32_timings.json', 'bench_read_f32_as_i16_timings.json']


In [6]:
def get_data(file, multiplier=1, dataset_size=20):
    with open(file) as f:
        data = json.load(f)
    data = data['benchmarks']
    rows = {}
    for bench in data:
        metadata = bench['metadata']
        bench_name = metadata['name']
        runs = bench['runs']
        run_means = []
        run_stds = []
        for run in runs:
            values = np.array(run['values'])
            mean = np.mean(values)
            std = np.std(values)
            run_means.append(mean)
            run_stds.append(std)
        mean = np.mean(np.array(run_means)) * multiplier
        dataset_mean = mean * dataset_size
        std = np.mean(np.array(run_stds)) * multiplier
        dataset_std = std * dataset_size
        rows[bench_name]=  {'mean': mean,'dataset_mean':dataset_mean, 'std': std, 'dataset_std': dataset_std}
    return rows

## Read Timings

In [10]:
timing_dfs = []
for file in r_timings:
    fp = os.path.join(results_dir, 'reading', file)
    rows = get_data(fp, multiplier=1000, dataset_size=20)
    # print(rows)
    timing_dfs.append(pd.DataFrame().from_dict(rows, orient='index'))
for df in timing_dfs:
    display(df)


,mean,dataset_mean,std,dataset_std
ScipyIO-i16,0.719620,14.392390,0.092770,1.855390
TorchAudio-i16-Sox,8.531971,170.639421,0.431617,8.632331
TorchAudio-i16-Soundfile,5.849088,116.981768,0.613085,12.261704
PyWavers-i16,10.720131,214.402618,0.641290,12.825804
Soundfile-i16,1.294757,25.895131,0.115172,2.303447


,mean,dataset_mean,std,dataset_std
ScipyIO-i16_as_f32_,0.785198,15.703959,0.094729,1.894580
TorchAudio-i16_as_f32_-Sox,5.601734,112.034687,0.461506,9.230124
TorchAudio-i16_as_f32_-Soundfile,3.587908,71.758169,0.184948,3.698964
PyWavers-i16_as_f32_,10.679072,213.581446,0.381531,7.630629
Soundfile-i16_as_f32_,3.234100,64.681992,0.085103,1.702062


,mean,dataset_mean,std,dataset_std
ScipyIO-f32_as_i16,1.073435,21.468694,0.165829,3.316582
TorchAudio-f32_as_i16-Sox,9.450603,189.012061,0.553205,11.064109
TorchAudio-f32_as_i16-Soundfile,3.312607,66.252146,0.551012,11.020236


In [14]:
tracemalloc_dfs = []
for file in r_mallocs:
    fp = os.path.join(results_dir, 'reading', file)
    rows = get_data(fp, multiplier=1, dataset_size=20)
    tracemalloc_dfs.append(pd.DataFrame().from_dict(rows, orient='index'))
for df in tracemalloc_dfs:
    df['mean'] = df['mean'] / 1024 / 1024
    df['dataset_mean'] = df['dataset_mean'] / 1024 / 1024
    display(df)


,mean,dataset_mean,std,dataset_std
ScipyIO-i16_as_f32_,13.292354,265.847089,0.0,0.0
TorchAudio-i16_as_f32_-Sox,69.785486,1395.709720,0.0,0.0
TorchAudio-i16_as_f32_-Soundfile,69.817994,1396.359887,0.0,0.0
PyWavers-i16_as_f32_,8.350247,167.004933,0.0,0.0
Soundfile-i16_as_f32_,8.616168,172.323358,0.0,0.0


,mean,dataset_mean,std,dataset_std
ScipyIO-i16,13.207182,264.143648,0.0,0.0
TorchAudio-i16-Sox,69.785634,1395.712681,0.0,0.0
TorchAudio-i16-Soundfile,69.814850,1396.297002,0.0,0.0
PyWavers-i16,8.350336,167.006721,0.0,0.0
Soundfile-i16,8.609555,172.191100,0.0,0.0


,mean,dataset_mean,std,dataset_std
ScipyIO-f32_as_i16,13.301141,266.022818,0.0,0.0
TorchAudio-f32_as_i16-Sox,69.785771,1395.715418,0.0,0.0
TorchAudio-f32_as_i16-Soundfile,69.817383,1396.347656,0.0,0.0


In [12]:
memory_dfs = []
for file in r_memory:
    fp = os.path.join(results_dir, 'reading', file)
    rows = get_data(fp, multiplier=1, dataset_size=20)
    tracemalloc_dfs.append(pd.DataFrame().from_dict(rows, orient='index'))
for df in tracemalloc_dfs:
    display(df)


,mean,dataset_mean,std,dataset_std
ScipyIO-i16_as_f32_,1.393804e+07,2.787609e+08,0.0,0.0
TorchAudio-i16_as_f32_-Sox,7.317539e+07,1.463508e+09,0.0,0.0
TorchAudio-i16_as_f32_-Soundfile,7.320947e+07,1.464189e+09,0.0,0.0
PyWavers-i16_as_f32_,8.755868e+06,1.751174e+08,0.0,0.0
Soundfile-i16_as_f32_,9.034707e+06,1.806941e+08,0.0,0.0


,mean,dataset_mean,std,dataset_std
ScipyIO-i16,13848734.50,2.769747e+08,0.0,0.0
TorchAudio-i16-Sox,73175541.00,1.463511e+09,0.0,0.0
TorchAudio-i16-Soundfile,73206176.25,1.464124e+09,0.0,0.0
PyWavers-i16,8755962.00,1.751192e+08,0.0,0.0
Soundfile-i16,9027772.75,1.805555e+08,0.0,0.0


,mean,dataset_mean,std,dataset_std
ScipyIO-f32_as_i16,1.394726e+07,2.789451e+08,0.0,0.0
TorchAudio-f32_as_i16-Sox,7.317568e+07,1.463514e+09,0.0,0.0
TorchAudio-f32_as_i16-Soundfile,7.320883e+07,1.464177e+09,0.0,0.0


,mean,dataset_mean,std,dataset_std
ScipyIO-i16_as_f32_,40025600.0,8.005120e+08,0.0,0.0
TorchAudio-i16_as_f32_-Sox,332504576.0,6.650092e+09,0.0,0.0
TorchAudio-i16_as_f32_-Soundfile,329209344.0,6.584187e+09,0.0,0.0
PyWavers-i16_as_f32_,28396032.0,5.679206e+08,0.0,0.0
Soundfile-i16_as_f32_,27330560.0,5.466112e+08,0.0,0.0


,mean,dataset_mean,std,dataset_std
ScipyIO-f32_as_i16,39989760.0,7.997952e+08,0.0,0.0
TorchAudio-f32_as_i16-Sox,333792768.0,6.675855e+09,0.0,0.0
TorchAudio-f32_as_i16-Soundfile,331407872.0,6.628157e+09,0.0,0.0


,mean,dataset_mean,std,dataset_std
ScipyIO-i16,39884288.0,7.976858e+08,0.0,0.0
TorchAudio-i16-Sox,333515776.0,6.670316e+09,0.0,0.0
TorchAudio-i16-Soundfile,331424768.0,6.628495e+09,0.0,0.0
PyWavers-i16,28431872.0,5.686374e+08,0.0,0.0
Soundfile-i16,27247104.0,5.449421e+08,0.0,0.0
